
# Ran gather with decontaminated db. 
- Found 19 spp in 99% of genomes. 
- Extracted marker genes with checkM 
- How many of the metaGs have a signal of the  marker genes
- predict CDS using prokka: How many metaGs have the CDS signal
- manysearch x pangenomes from GTDB
- manysearch x pangenomes from GTDB+MAGs


In [1]:
# imports
import pandas as pd


In [68]:
# open df 
dfgather = pd.read_csv('/group/ctbrowngrp2/amhorst/2025-pig-corespp/results/csv_files/251201_99core.10000kb.csv', names=["query_name", "gather_GTDBMAG"], header=0)
dfgather["query_name"] = dfgather["query_name"].str.replace(r"^.*s__\s*", "", regex=True)
dfgather["query_name"] = dfgather["query_name"].str.replace("_", " ")
ids = pd.read_csv("/group/ctbrowngrp2/amhorst/2025-pig-corespp/resources/metag-wort-hq.3217.txt", header=None)[0]



In [69]:
dfgather

,query_name,gather_GTDBMAG
0,Phascolarctobacterium A succinatutens,3189
1,Cryptobacteroides sp900546925,3190
2,Oscillibacter segnis,3183
3,JALFVM01 sp022787145,3187
4,Bariatricus sp004560705,3199
5,Gemmiger qucibialis,3185
6,Mogibacterium A kristiansenii,3197
7,Phocaeicola vulgatus,3193
8,UBA2868 sp004552595,3204
9,Holdemanella porci,3189


In [70]:
def open_df(path_csv, contain_val, sig_type):
    df = pd.read_csv(path_csv)
    df = df[['match_name', 'query_name', 'max_containment']]
    df = df[df["max_containment"] >= contain_val]
    df = df[df["match_name"].isin(ids)]
    df["query_name"] = df["query_name"].str.replace(r".*/", "", regex=True)
    df["query_name"] = df["query_name"].str.split(".").str[0]
    df["query_name"] = df["query_name"].str.replace("_", " ")
    counts = df["query_name"].value_counts().rename(sig_type)
    return df, counts

In [71]:
containment_threshold = 0.01
df_marker, c_marker = open_df('/group/ctbrowngrp2/amhorst/2025-pig-corespp/results/checkm/251204_manysearch.3216.marker.csv', containment_threshold, "marker_gene")
df_cds, c_cds = open_df('/group/ctbrowngrp2/amhorst/2025-pig-corespp/results/prokka/251204_manysearch.3216.CDS.csv', containment_threshold, "CDS")
df_ms_gtdb, c_gtdb = open_df('/group/ctbrowngrp2/amhorst/2025-pig-corespp/results/pangenome/251204_manysearch.3216.gtdb.csv',containment_threshold, "GTDB")
df_ms_gtdmag, c_gtdb_mag = open_df('/group/ctbrowngrp2/amhorst/2025-pig-corespp/results/pangenome/251204_manysearch.3216.gtdb+mags.csv',containment_threshold, "MAG+GTDB")

In [72]:
df_counts = pd.concat([c_marker, c_cds, c_gtdb, c_gtdb_mag], axis=1).fillna(0).astype(int)
df_counts = df_counts.reset_index().rename(columns={"index": "microbe"})
df_counts

,query_name,marker_gene,CDS,GTDB,MAG+GTDB
0,Lactobacillus amylovorus,3208,3205,3188,3206
1,Oscillibacter segnis,3187,3189,3189,3189
2,Phascolarctobacterium A succinatutens,3187,3197,3198,3197
3,Fimisoma sp002320005,3181,3195,3194,3196
4,Sodaliphilus sp004557565,3179,3183,3184,3184
5,Prevotella sp000434975,3176,3183,3183,3185
6,UBA2868 sp004552595,3171,3194,3132,3194
7,Prevotella sp002251295,3170,3187,3169,3186
8,Gemmiger qucibialis,3165,3192,3191,3192
9,Cryptobacteroides sp900546925,3165,3186,3186,3187


In [73]:
dfgather

,query_name,gather_GTDBMAG
0,Phascolarctobacterium A succinatutens,3189
1,Cryptobacteroides sp900546925,3190
2,Oscillibacter segnis,3183
3,JALFVM01 sp022787145,3187
4,Bariatricus sp004560705,3199
5,Gemmiger qucibialis,3185
6,Mogibacterium A kristiansenii,3197
7,Phocaeicola vulgatus,3193
8,UBA2868 sp004552595,3204
9,Holdemanella porci,3189


In [74]:
df = dfgather.merge(
    df_counts,on="query_name"
)

In [75]:
df

,query_name,gather_GTDBMAG,marker_gene,CDS,GTDB,MAG+GTDB
0,Phascolarctobacterium A succinatutens,3189,3187,3197,3198,3197
1,Cryptobacteroides sp900546925,3190,3165,3186,3186,3187
2,Oscillibacter segnis,3183,3187,3189,3189,3189
3,JALFVM01 sp022787145,3187,3133,3195,3178,3195
4,Bariatricus sp004560705,3199,3132,3193,3195,3195
5,Gemmiger qucibialis,3185,3165,3192,3191,3192
6,Mogibacterium A kristiansenii,3197,3154,3195,3146,3197
7,Phocaeicola vulgatus,3193,3130,2580,2796,2846
8,UBA2868 sp004552595,3204,3171,3194,3132,3194
9,Holdemanella porci,3189,3155,3192,3185,3193
